In [1]:
from bs4 import BeautifulSoup 
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from urllib.parse import urljoin
import requests
import pandas as pd


/Users/aravpatel/python projects/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
#setting up a Selenium browser
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")  #Anti-detection
driver = uc.Chrome(options=options)

Going to the section in the Hindu about the delhi elections. 

In [71]:
mainlink = "https://www.thehindu.com/elections/delhi-assembly/"
driver.get(mainlink)


In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser') # Souping it to identify the structure of the webpage
print(soup.prettify())

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <script async="" src="https://connect.facebook.net/signals/config/2498379470430398?v=2.9.184&amp;r=stable&amp;domain=www.thehindu.com&amp;hme=93c44828b638838a88daf3a14a90ff297f7fc4ade758f21de5500c9f17542765&amp;ex_m=70%2C122%2C107%2C111%2C61%2C4%2C100%2C69%2C16%2C97%2C89%2C51%2C54%2C173%2C176%2C188%2C184%2C185%2C187%2C29%2C101%2C53%2C77%2C186%2C168%2C171%2C181%2C182%2C189%2C132%2C41%2C194%2C191%2C192%2C34%2C144%2C15%2C50%2C198%2C197%2C134%2C18%2C40%2C1%2C43%2C65%2C66%2C67%2C71%2C93%2C17%2C14%2C96%2C92%2C91%2C108%2C52%2C110%2C39%2C109%2C30%2C94%2C26%2C169%2C172%2C141%2C86%2C56%2C84%2C33%2C73%2C0%2C95%2C32%2C28%2C82%2C83%2C88%2C47%2C46%2C87%2C37%2C11%2C12%2C13%2C6%2C7%2C25%2C22%2C23%2C57%2C62%2C64%2C75%2C102%2C27%2C76%2C9%2C8%2C80%2C48%2C21%2C104%2C103%2C105%2C98%2C10%2C20%2C3%2C38%2C74%2C19%2C5%2C90%2C81%2C44%2C35%2C85%2C2%2C36%2C63%2C42%2C106%2C45%2C79%2C68%2C112%2C60%2C59%2C31%2C99%2C58%2C55%2C49%2C78%2C72%2

The files link belong in the class title

In [ ]:
def extract_links(soup):
    # All the articles are in h3 tags of either class title or title big
    articles = soup.find_all("h3", class_=["title", "title big"])
    
    # Extract links
    links = [article.find("a")["href"] for article in articles if article.find("a")]
    
    return links

extracts all links from the page and goes to the next one once its done.

In [ ]:
news_links = []
add = "?page=" #add the page number to the main link to go to the next page
for i in range(1, 10):
    driver.get(mainlink + add + str(i))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    news_links.extend(extract_links(soup))
    time.sleep(2)
    if (len(news_links) > 150): #break if there are more than 150 links
        break

In [7]:
print(len(news_links))

139


In [ ]:
#remove any duplicate links 
news_links = list(set(news_links))
print(len(news_links))

138


In [ ]:
df = pd.DataFrame(news_links, columns=["links"])
df.to_csv("links_all.csv", index=False) #save all links to a csv file


In [19]:
news_links = pd.read_csv("links_all.csv")
news_links = news_links["links"].tolist()


In [ ]:
#souping an example article to identify the structure.
link1 = news_links[2]
driver.get(link1)
soup = BeautifulSoup(driver.page_source, 'html.parser')
print(soup.prettify())

<html itemscope="" itemtype="http://schema.org/NewsArticle" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Atishi, relatives of Bidhuri booked for poll code violation in Kalkaji - The Hindu
  </title>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="Atishi, relatives of Bidhuri booked for poll code violation in Kalkaji" name="title"/>
  <meta content="Delhi police file FIR against AAP candidate Atishi for MCC violation, sparking controversy ahead of Assembly polls." name="description"/>
  <meta content="2025-02-05T01:40:20+05:30" name="publish-date"/>
  <meta content="2025-02-05T01:40:20+05:30" name="created-date"/>
  <meta content="2025-02-05T02:29:46+05:30" name="modified-date"/>
  <meta content="2025-02-04T20:10:20.000Z" property="article:published_time"/>
  <meta content="2025-02-04T20:59:46Z" property="article:modified_time"/

In [ ]:
def extract_title(soup): #function to extract the title of the article
    title = soup.find("h1", class_="title").text
    return title

In [ ]:
ignore = ["referpara", "related-topics-list", "publish-time-new"] #ignores common classes that link to other articles
def extract_article(soup): #function to extract the text of an article
    # Locate the main article container
    article_container = soup.find("div", class_="articlebodycontent")

    if not article_container:
        print("Could not find article")
        return None

    # Extract all <p> tags inside this container, excluding the unwanted classes
    paragraphs = article_container.find_all("p", class_=lambda c: c not in ignore)

    article_text = []
    for p in paragraphs:
        text = " ".join(p.stripped_strings) 
        article_text.append(text)

    # Join all paragraphs with double line breaks for readability
    return "\n\n".join(article_text)


In [63]:
article_title = extract_title(soup)
print(article_title + "\n")

article_content = extract_article(soup)
print(article_content)



Inflation, civic amenities, representation, and safety: women voters in Delhi talk about issues that will decide their ballot


As she drives her e-rickshaw through the streets of south Delhi’s Khanpur, ignoring the stares of passersby, Savitri, 49, says she would like Delhi to be a more inclusive city for women. She adds that better roads and improved law and order will be on her mind when she goes out to vote on Wednesday (February 5).

Inflation, too, is a pressing concern for her. “Free bus rides help me because I travel from Khanpur to the auto stand in Nehru Place every day,” she says. However, the promises of monthly cash transfer schemes announced by the three major political parties of the Capital — AAP (₹2,100), BJP (₹2,500), and Congress (₹2,500) — don’t appeal to her as she is sceptical about their implementation.

But for Lajjawati, 38, a domestic help who lives in Kusumpur Pahari, a slum cluster close to the affluent Vasant Kunj, the promise of ₹2,000 a month is a key is

Scraping the article and title from each link. I have used seperate lists for both instead of a dictionary since that was leading to formating issues

In [ ]:
article_data = []
article_titles = []

for i,link in enumerate(news_links):
    print(f"Scraping article {i+1}/{len(news_links)}")
    try:
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        article_title = extract_title(soup)
        if article_title:
            article_titles.append(article_title)
        article_text = extract_article(soup)
        if article_text:
            article_data.append(article_text)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        continue

Scraping article 1/138
Scraping article 2/138
Scraping article 3/138
Scraping article 4/138
Scraping article 5/138
Scraping article 6/138
Scraping article 7/138
Scraping article 8/138
Scraping article 9/138
Scraping article 10/138
Scraping article 11/138
Scraping article 12/138
Scraping article 13/138
Scraping article 14/138
Scraping article 15/138
Scraping article 16/138
Scraping article 17/138
Scraping article 18/138
Scraping article 19/138
Scraping article 20/138
Scraping article 21/138
Scraping article 22/138
Scraping article 23/138
Scraping article 24/138
Scraping article 25/138
Scraping article 26/138
Scraping article 27/138
Scraping article 28/138
Scraping article 29/138
Scraping article 30/138
Scraping article 31/138
Scraping article 32/138
Scraping article 33/138
Scraping article 34/138
Scraping article 35/138
Scraping article 36/138
Scraping article 37/138
Scraping article 38/138
Scraping article 39/138
Scraping article 40/138
Scraping article 41/138
Scraping article 42/138
S

In [ ]:

print(len(article_titles))
print(len(article_data)) #confirming that they are of the same length and thus unique


138
138


I have exported all the articles into an html doc to get a better look of the data.

In [115]:
import os

# Generate HTML content
html_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Scraped Articles</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 20px;
            background-color: #f4f4f4;
        }
        .container {
            max-width: 900px;
            margin: auto;
            background: white;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .article {
            margin-bottom: 30px;
            padding-bottom: 15px;
            border-bottom: 2px solid #ddd;
        }
        .article a {
            font-size: 18px;
            font-weight: bold;
            text-decoration: none;
            color: #333;
        }
        .article a:hover {
            color: #007bff;
        }
        .content {
            margin-top: 10px;
            font-size: 16px;
            color: #444;
            white-space: pre-wrap; /* Preserve new lines */
            background: #f9f9f9;
            padding: 10px;
            border-radius: 5px;
            overflow-y: auto;
            max-height: 300px; /* Scroll if too long */
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Scraped Articles</h1>
"""

# Loop through each article and add it to the HTML
for article in article_data:
    html_content += f"""
        <div class="article">
            <a href="#">{article_titles[article_data.index(article)]}</a>
            <div class="content">{article}</div>
        </div>
    """

html_content += """
    </div>
</body>
</html>
"""

# Save to an HTML file
output_file = "scraped_articles.html"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(html_content)


In [77]:
os.system(f"open {output_file}") 

0

I have all the articles into a txt to clean up any non-recurring redunancies such as article links, sources, etc.

In [ ]:
filename = "to_clean.txt"

with open(filename, "w", encoding="utf-8") as file:
    for idx, article in enumerate(article_data):
        file.write(f"Article {idx + 1}\n")
        file.write(article)
        file.write("\n\n")
        file.write("\n" + "-" * 80 + "\n\n")  # Add a Separator between the articles


In [ ]:
#loading the txt file after it was cleaned
import re 
filename = "cleaned.txt"

with open(filename, "r", encoding="utf-8") as file:
    content = file.read()

# Split using the separator
article_list = content.split("\n" + "-" * 80 + "\n\n")
article_list = [article.strip() for article in article_list if article.strip()]
article_data_cleaned = [re.sub(r"^Article \d+\s*", "", article, flags=re.IGNORECASE).strip() for article in article_list]

print(f"Loaded {len(article_data_cleaned)} articles")




Loaded 138 articles


In [128]:
print(article_data_cleaned[100])

On the eve of the counting of votes for the Delhi Assembly election, Aam Aadmi Party chief Arvind Kejriwal on Friday alleged that the Election Commission (EC) refused to upload Form 17C and the number of votes polled per booth in each Assembly constituency on its official website despite repeated requests.

Form 17C records the number of votes cast at each polling booth and is signed by the presiding officer and polling agents.

The AAP chief said his party has created a website to share booth-wise voting data from Delhi. “AAP has launched a website – http://transparentelections.in – where we have uploaded the Form 17Cs of every Assembly. This form has all the details of votes polled in each booth. We will present the data for every Assembly and booth in a tabulated format so that every voter can access this information. This is something that the EC should have done in the interest of transparency, but, unfortunately, they are refusing to do it,” Mr. Kejriwal said in a post on X.

Res

 <p class="referpara">
 <p class="related-topics-list">
 <p class="publish-time-new">



Saving a new clean html file with all the articles to have a better look.

In [119]:

# Loop through each article and add it to the HTML
html_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Scraped Articles</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 20px;
            background-color: #f4f4f4;
        }
        .container {
            max-width: 900px;
            margin: auto;
            background: white;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .article {
            margin-bottom: 30px;
            padding-bottom: 15px;
            border-bottom: 2px solid #ddd;
        }
        .article a {
            font-size: 18px;
            font-weight: bold;
            text-decoration: none;
            color: #333;
        }
        .article a:hover {
            color: #007bff;
        }
        .content {
            margin-top: 10px;
            font-size: 16px;
            color: #444;
            white-space: pre-wrap; /* Preserve new lines */
            background: #f9f9f9;
            padding: 10px;
            border-radius: 5px;
            overflow-y: auto;
            max-height: 300px; /* Scroll if too long */
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Scraped Articles</h1>
"""
for article in article_data_cleaned:
    html_content += f"""
        <div class="article">
            <a href="#">{article_titles[article_data_cleaned.index(article)]}</a>
            <div class="content">{article}</div>
        </div>
    """

html_content += """
    </div>
</body>
</html>
"""

# Save to an HTML file
output_file = "scraped_articles_clean.html"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(html_content)


In [124]:
#save article_data and news_links as a csv
df = pd.DataFrame({"links": news_links, "articles": article_data, "titles": article_titles})
#size of df
print(df.shape)
df.to_csv("cleaned_articles.csv", index=False)

(138, 3)
